# Automated Portfolio Builder using S&P 500 Companies

 - We are using Markowitz Theory to build profitable portfolio's from the S&P 500 Companies by taking consideration the user's investment capital and interested sectors.

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime

import iexfinance
from iexfinance.stocks import Stock
from iexfinance.stocks import get_historical_data

#import panel as pn

import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

### Pre-load Data Collection

In [2]:
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df = df[df['Date first added']<'2016-01-01']
df = df[df['Founded']<'2016']

sectors = pd.DataFrame()
sectors["Company"] = df["Symbol"]
sectors["Sector"] = df["GICS Sector"]

t = df["Symbol"].unique()
sec = df["GICS Sector"].unique()

def getHistoricalPrices(stock):
    start = datetime(2016, 1, 1)
    end = datetime.now()
    f = open("token.txt",'r')
    f = f.read()
    df = pd.DataFrame()
    for a in stock:
        try:
            tf = get_historical_data(a, start, end, token=f)
            df[a] = tf['close']  
        except (iexfinance.utils.exceptions.IEXQueryError, iexfinance.utils.exceptions.IEXSymbolError) as e:
            pass
    return df

data = getHistoricalPrices(t)

### Logic

In [3]:
def get_cx_port(invest,cx_sec,cx_risk,years):
    cx_risk = cx_risk/100
    df = []
    cx_sym = []
    for a in cx_sec:
        df.append(sectors[sectors['Sector'] == a])
    
    for x in range(len(cx_sec)):
        cx_sym.append(df[x]['Company'])

    cx_data = pd.DataFrame()
    for a in cx_sym:
        cx_data[a] = data[a]

    cx_data = (cx_data/cx_data.iloc[0]*100)
    cx_returns = []
    cx_volatilities = []
    cx_weights = []
    in_returns = []
    log_returns = np.log(cx_data/cx_data.shift(1))
    log_returns = log_returns.dropna()
    for x in range(2000):
        weights = np.random.random(len(cx_data.columns))
        weights /= np.sum(weights)
        cx_weights.append(weights)
        in_returns.append(invest*(1+(np.sum(weights*log_returns.mean()) * 250))**years)
        cx_returns.append(np.sum(weights*log_returns.mean()) * 250)
        cx_volatilities.append(np.sqrt(np.dot(weights.T,np.dot(log_returns.cov()*250,weights))))

    cx_weights = pd.DataFrame(cx_weights)
    cx_weights.columns = cx_data.columns
    cx_weights = cx_weights*invest
    cx_weights = cx_weights.round(0)
    cx_returns = np.array(cx_returns)
    cx_volatilities = np.array(cx_volatilities)
    in_returns = np.array(in_returns)
    in_returns = np.round(in_returns,0)
    
    portfolios = pd.DataFrame({'Returns':cx_returns,'Volatility':cx_volatilities,'In '+str(years)+' Years':in_returns})
    portfolios = pd.concat([portfolios,cx_weights], axis=1, sort=False)
    
    min_risk = min(portfolios['Volatility'])
    opt_port = portfolios[portfolios['Volatility']==min_risk]
    min_return = float(opt_port['Returns'])
    cx_port = portfolios[portfolios['Volatility'] > min_risk]
    cx_port = cx_port[cx_port['Returns'] > min_return]
    cx_port = cx_port[cx_port['Volatility'] < cx_risk]
    cx_port = cx_port.round(3)
    cx_port = cx_port.sort_values(by='Returns', ascending=False)
    
    return cx_port

### Dashboard

In [ ]:
app = dash.Dash(__name__)
app.config.suppress_callback_exceptions = True
#app.serverapp.config.suppress_callback_exceptions = True
app.layout = html.Div([
    html.Div([
        html.H1('Automated Portfolio Builder for S&P 500 Comapanies'),
        html.Hr()
    ]),
    html.Div([html.P(),
        html.H4('Enter the amount you like to invest'),
        dcc.Input(id='invest',
                  type='number',
                  value='')
    ]),
    html.Div([html.P(),
    html.H4('Select the Sectors you want to invest in'),
    
    dcc.Dropdown(id = 'sectors'
                        ,options=[
                             {'label': i, 'value': i} for i in sec],
                        value=[],
                        multi=True)
    ]),
    html.Div([html.P(),
    html.H4('Enter the maximum Risk % you can take'),
    dcc.Input(id='risk',
              type='number',
              value='')
    ]),
    html.Div([html.P(),
        html.H4('For how many years you want to make this investment'),
        dcc.Input(id='years',
                  type='number',
                  value='')    
    ]),
    html.Div([html.P(),
    html.Button(id='submit', type='button', children='ok')
    ]),
    html.Div([html.P(),
        html.H3(children='Portfolios'), 
        html.Table(id='my_table')], 
        style={'width': '90%','display': 'inline-block', 'padding': '0 20'})
  ])

@app.callback(
    Output("my_table", "children"),
    [Input("submit","n_clicks")],
    state=[State("invest","value"),
    State("sectors","value"),
    State("risk", "value"),
    State("years","value")])
def return_clicks(n_clicks,invest,sectors,risk,years):
    dataframe = get_cx_port(invest,sectors,risk,years)
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns]) ] +
        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range((len(dataframe)))]
    )
    
if __name__ == "__main__":
        app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
